###  Module & Utility Imports

In [1]:
import os
import re
import pandas as pd
import numpy as np

from Utilities.EvaluationMain import *
from Utilities.Utilities import ReadYaml, SerializeObjects, DeserializeObjects, LoadModelConfigs, LoadParams
from Models.Caller64 import *
from Utilities.Visualization import VisReconGivenZ_FCA, HeatMapFreqZ_FCA, VisReconGivenFC_ZA, VisReconExtractZ_FC

### Data Loading: Model Configs and Evaluation Results

In [2]:
def load_evaluation_tables(directory, acc_keyword, acc_pattern, mi_keyword, mi_pattern):
    """Load and combine evaluation tables from directory based on filtering keywords."""
    table_list = os.listdir(directory)
    
    # Load accuracy tables
    acc_list = [tab for tab in table_list if acc_keyword in tab and acc_pattern in tab]
    acc_df = pd.DataFrame()
    for tab in acc_list:
        file_path = os.path.join(directory, tab)
        df = pd.read_csv(file_path)
        acc_df = pd.concat([acc_df, df], axis=0)
    
    # Compute RMSE from MSE
    if 'MSEdenorm' in acc_df.columns:
        acc_df['RMSE'] = np.sqrt(acc_df['MSEdenorm'])
    
    # Load MI tables
    mi_list = [tab for tab in table_list if mi_keyword in tab and mi_pattern in tab]
    mi_df = pd.DataFrame()
    for tab in mi_list:
        file_path = os.path.join(directory, tab)
        df = pd.read_csv(file_path)
        mi_df = pd.concat([mi_df, df], axis=0)
    
    return acc_df, mi_df

def load_config_models(config_directory, include_keyword='Config', exclude_keyword='Eval', key='Models'):
    """Extract model configurations from YAML files."""
    config_files = [f for f in os.listdir(config_directory)
                    if include_keyword in f and exclude_keyword not in f]
    model_dict = {}
    for config in config_files:
        full_path = os.path.join(config_directory, config)
        config_data = ReadYaml(full_path)
        model_dict[config.split('.')[0]] = list(config_data.get(key, {}).keys())
    return model_dict

# Load evaluation tables
eval_directory = './EvalResults/Tables/'
AcctableSet, MItableSet = load_evaluation_tables(
    eval_directory,
    acc_keyword='Acc',
    acc_pattern='Nj1_FC',
    mi_keyword='MI',
    mi_pattern='Nj1_FC')

# Load benchmark tables
bench_directory = './Benchmarks/EvalResults/Tables/'
BenchAcctableSet, BenchMItableSet = load_evaluation_tables(
    bench_directory,
    acc_keyword='Acc',
    acc_pattern='NjAll',
    mi_keyword='MI',
    mi_pattern='NjAll')

# Unify metric naming conventions
metrics_map = {
    '(i) $I(V;\\acute{\\Theta} \\mid X)$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(ii) $I(S;\\acute{\\Theta} \\mid X)$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$'}

BenchMItableSet['Metrics'] = BenchMItableSet['Metrics'].replace(metrics_map)
MetricTypes = np.unique(MItableSet['MetricType']).tolist()

In [11]:
BenchMItableSet

,Model,Metrics,Values,MetricType
0,ConVAE_ART_30_Mimic,(i) $I(V; \acute{Z} \mid Z)$,0.047120,fft
1,ConVAE_ART_30_Mimic,(i) $I(V; \acute{Z} \mid Z)$,0.054335,fft
2,ConVAE_ART_30_Mimic,(i) $I(V; \acute{Z} \mid Z)$,0.049179,fft
3,ConVAE_ART_30_Mimic,(i) $I(V; \acute{Z} \mid Z)$,0.041711,fft
4,ConVAE_ART_30_Mimic,(i) $I(V; \acute{Z} \mid Z)$,0.069489,fft
...,...,...,...,...
1195,Wavenet_II_VitalDB,(iii) $I(S;\acute{\Theta} \mid \acute{Z})$,0.117728,matching_pursuit
1196,Wavenet_II_VitalDB,(iii) $I(S;\acute{\Theta} \mid \acute{Z})$,0.117771,matching_pursuit
1197,Wavenet_II_VitalDB,(iii) $I(S;\acute{\Theta} \mid \acute{Z})$,0.114526,matching_pursuit
1198,Wavenet_II_VitalDB,(iii) $I(S;\acute{\Theta} \mid \acute{Z})$,0.107739,matching_pursuit


### Functions

In [3]:
def load_config_models(config_directory, include_keyword='Config', exclude_keyword='Eval', key='Models'):
    """Extract model configurations from YAML files."""
    config_files = [f for f in os.listdir(config_directory)
                    if include_keyword in f and exclude_keyword not in f]
    model_dict = {}
    for config in config_files:
        full_path = os.path.join(config_directory, config)
        config_data = ReadYaml(full_path)
        model_dict[config.split('.')[0]] = list(config_data.get(key, {}).keys())
    return model_dict

def prepare_analysis_table(mi_df, acc_df, target_models):
    """
    Merge MI and accuracy data, compute composite ISCORE metrics, and parse model parameters.
    """
    # Filter by target models
    mi_table = mi_df[mi_df['Model'].isin(target_models)].reset_index(drop=True)
    acc_table = acc_df[acc_df['Model'].isin(target_models)].reset_index(drop=True)
    
    # Prepare accuracy table
    if 'MAPEnorm' in acc_table.columns:
        acc_table['MAPEnorm'] = acc_table['MAPEnorm'] / 100
    acc_table = acc_table[['Model', 'MeanKldRes', 'RMSE', 'R2denorm']].copy()
    acc_table.columns = ['Model', 'FQI', 'RMSE', 'R2denorm']
    
    # Process MI table: group by Model and Metrics, then pivot
    mi_grouped = mi_table.groupby(['Model', 'Metrics']).mean(numeric_only=True).reset_index()
    mi_pivot = pd.pivot(mi_grouped, index='Model', columns='Metrics', values='Values').reset_index()
    
    # Merge tables
    merged_table = pd.merge(mi_pivot, acc_table, on='Model', how='inner').sort_values('Model').reset_index(drop=True)
    
    # Parse model parameters from naming convention
    split_cols = merged_table['Model'].str.split('_', expand=True)
    if split_cols.shape[1] == 6:
        split_cols.columns = ['Prefix', 'Type', 'Depth', 'LatDim', 'Comp', 'Source']
        merged_table = pd.concat([merged_table, split_cols], axis=1)
    elif split_cols.shape[1] == 4:
        mask = split_cols[3].isna() | (split_cols[3] == 'None')
        split_cols.loc[mask, 3] = split_cols.loc[mask, 2]
        split_cols.loc[mask, 2] = 0
        split_cols.columns = ['Prefix', 'Type', 'LatDim', 'Source']
        merged_table = pd.concat([merged_table, split_cols], axis=1)
    else:
        print("Warning: Unexpected model naming format. Check the 'Model' column.")
    
    # Compute individual performance metrics
    ms_metric_col = '(i) $I(V; \\acute{Z} \\mid Z)$'
    
    merged_table['MP'] = 1-np.sqrt(1 - np.exp(-2 * merged_table['(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$']))
    merged_table['AC'] = np.sqrt(1 - np.exp(-2 * merged_table['(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$']))
    merged_table['SS'] = 1-np.sqrt(1 - np.exp(-2 * merged_table['FQI']))
    merged_table['RA'] = merged_table['R2denorm']
    merged_table['NMSE'] = 1 - merged_table['RA']

    # Compute composite ISCORE (with/without MS metric)
    if ms_metric_col in merged_table.columns:
        mask_has_ms = merged_table[ms_metric_col].notna()
        merged_table.loc[mask_has_ms, 'MS'] = np.sqrt(1 - np.exp(-2 * merged_table.loc[mask_has_ms, ms_metric_col]))
        
        # ISCORE with MS
        merged_table.loc[mask_has_ms, 'ISCOREam'] = mean(merged_table.loc[mask_has_ms, ['MS', 'MP', 'AC', 'SS', 'RA']], kind="am", axis=1)
        merged_table.loc[mask_has_ms, 'ISCOREgm'] = mean(merged_table.loc[mask_has_ms, ['MS', 'MP', 'AC', 'SS', 'RA']], kind="gm", axis=1)
        merged_table.loc[mask_has_ms, 'ISCOREhm'] = mean(merged_table.loc[mask_has_ms, ['MS', 'MP', 'AC', 'SS', 'RA']], kind="hm", axis=1)
        
        # ISCORE without MS
        mask_no_ms = merged_table[ms_metric_col].isna()
        merged_table.loc[mask_no_ms, 'ISCOREam'] = mean(merged_table.loc[mask_no_ms, ['MP', 'AC', 'SS', 'RA']], kind="am", axis=1)
        merged_table.loc[mask_no_ms, 'ISCOREgm'] = mean(merged_table.loc[mask_no_ms, ['MP', 'AC', 'SS', 'RA']], kind="gm", axis=1)
        merged_table.loc[mask_no_ms, 'ISCOREhm'] = mean(merged_table.loc[mask_no_ms, ['MP', 'AC', 'SS', 'RA']], kind="hm", axis=1)
    else:
        merged_table['ISCOREam'] = mean(merged_table[['MP', 'AC', 'SS', 'RA']], kind="am", axis=1)
        merged_table['ISCOREgm'] = mean(merged_table[['MP', 'AC', 'SS', 'RA']], kind="gm", axis=1)
        merged_table['ISCOREhm'] = mean(merged_table[['MP', 'AC', 'SS', 'RA']], kind="hm", axis=1)
        
    return merged_table

def amean(x, weights=None, axis=None, eps=None):
    """Arithmetic mean (simple or weighted)."""
    if isinstance(x, (pd.DataFrame, pd.Series)):
        x = x.values

    x = np.asarray(x, dtype=float)

    if weights is None:
        return np.mean(x, axis=axis)

    w = np.asarray(weights, dtype=float)
    return np.average(x, weights=w, axis=axis)
    
def gmean(x, weights=None, axis=None, eps=1e-12):
    """Geometric mean with eps stabilization for zero values."""
    if isinstance(x, (pd.DataFrame, pd.Series)):
        x = x.values
    
    x = np.asarray(x, dtype=float)
    logs = np.log(x + eps)
    if weights is None:
        return np.exp(np.mean(logs, axis=axis))
    return np.exp(np.average(logs, weights=np.asarray(weights, dtype=float), axis=axis))

def hmean(x, weights=None, axis=None, eps=1e-12):
    """Harmonic mean with eps stabilization."""
    if isinstance(x, (pd.DataFrame, pd.Series)):
        x = x.values
    
    x = np.asarray(x, dtype=float)
    if weights is None:
        denom = np.sum(1.0 / (x + eps), axis=axis)
        count = x.size if axis is None else x.shape[axis]
        return count / denom
    w = np.asarray(weights, dtype=float)
    return np.sum(w, axis=axis) / np.sum(w / (x + eps), axis=axis)

def mean(x, kind="gm", weights=None, axis=None, eps=1e-12):
    """Unified interface for arithmetic, geometric, and harmonic means."""
    kind = kind.lower()
    if kind == "am":
        return amean(x, weights=weights, axis=axis)
    if kind == "gm":
        return gmean(x, weights=weights, axis=axis, eps=eps)
    if kind == "hm":
        return hmean(x, weights=weights, axis=axis, eps=eps)
    raise ValueError("kind must be one of {'am','gm','hm'}")

### Analysis Table Construction and Parameter Selection

In [4]:
# Load configuration models and combine all model keys
config_directory = './Config/'
TabLists = load_config_models(config_directory)
AnalTabList = list(np.concatenate([tabs for key, tabs in TabLists.items()]))

legend_map = {
    'Depth': r'$\zeta$',
    'LatDim': r'$J$',
    'Comp': r'$C$'
}

# Define main models (excluded from ablation analysis)
MainList = [
    'SKZFC_ART_1_30_800_Mimic',
    'SKZFC_ART_1_30_800_VitalDB',
    'SKZFC_II_1_30_800_Mimic',
    'SKZFC_II_1_30_800_VitalDB'
]

AnalMetricList = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$'
]

# Prepare merged analysis table by metric type
AnalAccMItableDic = {}
SubAcctableSet = AcctableSet.copy()
for mtype in MetricTypes:
    SubMItableSet = MItableSet[MItableSet['MetricType'] == mtype].reset_index(drop=True)
    KldCols = [col for col in AcctableSet.columns if 'MeanKld' in col]
    SelKldCols = [type for type in KldCols if mtype in type]
    SubAcctableSet['MeanKldRes'] = AcctableSet[SelKldCols]
    AnalAccMItableDic[mtype] = prepare_analysis_table(SubMItableSet, SubAcctableSet, AnalTabList)

AnalAccMItableMerged = pd.concat(AnalAccMItableDic, axis=0)
AnalAccMItableMerged = AnalAccMItableMerged.reset_index(level=0).rename(columns={'level_0': 'MetricType'})

# Create sensitivity analysis table (exclude main models, keep only SKZFC variants)
SenseAccMItable = AnalAccMItableMerged[~AnalAccMItableMerged['Model'].isin(MainList)].copy()
SenseAccMItable = SenseAccMItable[SenseAccMItable['Model'].str.contains('SKZFC', na=False)].reset_index(drop=True)
SenseAccMItable = SenseAccMItable[['MetricType','Model', 'Source', 'Type', 'Depth', 'LatDim', 'Comp', 'MS', 'MP', 'AC', 'SS', 'RA', 'ISCOREam', 'ISCOREgm', 'ISCOREhm']]

### Benchmark Analysis Table Construction

In [5]:
# Load benchmark configuration models
Bench_config_directory = './Benchmarks/Config/'
BenchTabLists = load_config_models(Bench_config_directory)
BenchAnalTabList = list(np.concatenate([tabs for key, tabs in BenchTabLists.items()]))

# Prepare benchmark analysis table by metric type
BenchAnalAccMItableDic = {}
SubBenchAcctableSet = BenchAcctableSet.copy()
for mtype in MetricTypes:
    SubBenchMItableSet = BenchMItableSet[BenchMItableSet['MetricType'] == mtype].reset_index(drop=True)
    KldCols = [col for col in BenchAcctableSet.columns if 'MeanKld' in col]
    SelKldCols = [type for type in KldCols if mtype in type]
    SubBenchAcctableSet['MeanKldRes'] = BenchAcctableSet[SelKldCols]
    BenchAnalAccMItableDic[mtype] = prepare_analysis_table(SubBenchMItableSet, SubBenchAcctableSet,  BenchAnalTabList)

BenchAnalAccMItableMerged = pd.concat(BenchAnalAccMItableDic)
BenchAnalAccMItableMerged = BenchAnalAccMItableMerged.reset_index(level=0).rename(columns={'level_0': 'MetricType'})

# Extract reference performance baselines
RAReferences = BenchAnalAccMItableMerged[['Type','Source', 'RA']].groupby(['Type','Source']).min().reset_index()

### Model Performance Aggregation and Sensitivity Analysis

In [6]:
# Aggregate performance across all methods
AllMethods = pd.concat(AnalAccMItableDic.values(), ignore_index=True)

# Keep SKZFC models only
SKZFCs = AllMethods[AllMethods['Model'].str.contains('SKZFC', na=False)].reset_index(drop=True)

# Filter out models below reference performance threshold
RemoveList = []
for idx, row in SKZFCs.iterrows():
    target_mask = (RAReferences['Type'] == row['Type']) & (RAReferences['Source'] == row['Source'])
    if row['RA'] < RAReferences[target_mask]['RA'].values[0]:
        RemoveList.append(row['Model'])
RemoveList = list(set(RemoveList))

# Compute mean ISCOREgm for each (Source, Type, Model)
group_cols = ['Source', 'Type', 'Model']
MetricType = ['ISCOREgm']
AggregatedScores = SKZFCs.groupby(group_cols)[MetricType].mean().reset_index()

# Exclude underperforming models
AggregatedScores_best = AggregatedScores[~AggregatedScores['Model'].isin(RemoveList)]

n_methods = len(AnalAccMItableDic)
n_metrics = len(MetricType)

# Select best model per (Type, Source)
best_model_indices = AggregatedScores_best.groupby(['Type', 'Source'])['ISCOREgm'].idxmax()
BestOverallModels = AggregatedScores_best.loc[best_model_indices, ['Type', 'Source', 'Model', 'ISCOREgm']].reset_index(drop=True)
BestModels = pd.merge(BestOverallModels[['Model']], AnalAccMItableMerged, on='Model').reset_index(drop=True)

# Unified sensitivity analysis function
def calculate_sensitivity(df, hp, score_col, group_by=('Type','Source'),
                          latex_labels=True, legend_map=None,
                          n_methods=None, n_metrics=None):
    """Compute sensitivity statistics for a given hyperparameter with LaTeX-ready labels."""
    import pandas as pd

    if hp not in df.columns:
        return None, None

    legend_map = legend_map or {}

    # Detailed statistics per (group_by..., hp)
    group_cols_detail = list(group_by) + [hp]
    detail_stats = (
        df.groupby(group_cols_detail)[score_col]
          .agg(mean='mean', std='std', max='max', min='min', n='size')
          .reset_index()
          .rename(columns={hp: 'Setting'})
    )

    # Build LaTeX display label for hyperparameter
    if latex_labels:
        if hp in legend_map and isinstance(legend_map[hp], str) and legend_map[hp].strip():
            hp_label = legend_map[hp]
        else:
            fallback = {'Depth': r'$\zeta$', 'LatDim': r'$J$', 'Comp': r'$C$'}
            hp_label = fallback.get(hp, hp)
            if isinstance(hp_label, str) and not (hp_label.startswith('$') and hp_label.endswith('$')):
                hp_label = rf'$\mathrm{{{hp_label}}}$'
    else:
        hp_label = legend_map.get(hp, hp)

    detail_stats['Hyperparameter'] = hp_label

    # Annotate effective data points
    if (n_methods is not None) and (n_metrics is not None):
        total_points = n_methods * n_metrics
        detail_stats['n_effective'] = detail_stats['n'] * total_points
        detail_stats['note'] = f'Each model aggregated from {n_methods}x{n_metrics}={total_points} data points'

    # Summary statistics per (group_by..., Hyperparameter)
    group_cols_summary = list(group_by) + ['Hyperparameter']
    summary_stats = (
        detail_stats.groupby(group_cols_summary)
        .agg(
            SensRange=('mean', lambda x: x.max() - x.min()),
            SensStd=('mean', 'std'),
            N_Settings=('Setting', 'nunique'),
            Total_Models=('n', 'sum')
        )
        .reset_index()
        .fillna(0)
    )

    if 'n_effective' in detail_stats.columns:
        tmp = (detail_stats.groupby(group_cols_summary)['n_effective']
               .sum().reset_index(name='Total_Effective_DataPoints'))
        summary_stats = summary_stats.merge(tmp, on=group_cols_summary, how='left')

    # Column ordering
    summary_cols = list(group_by) + ['Hyperparameter', 'SensStd', 'SensRange', 'N_Settings', 'Total_Models']
    if 'Total_Effective_DataPoints' in summary_stats.columns:
        summary_cols.append('Total_Effective_DataPoints')

    detail_cols = list(group_by) + ['Hyperparameter', 'Setting', 'mean', 'std', 'max', 'min', 'n']
    if 'n_effective' in detail_stats.columns:
        detail_cols += ['n_effective', 'note']

    return summary_stats[summary_cols], detail_stats[detail_cols]

# Aggregated sensitivity analysis
ParamCols = ['Depth', 'LatDim', 'Comp']

# Merge aggregated scores with hyperparameter columns
template_df = next(iter(AnalAccMItableDic.values()))
merge_cols = ['Source', 'Type', 'Model'] + ParamCols
template_df = template_df[template_df['Model'].str.contains('SKZFC', na=False)].reset_index(drop=True)
template_subset = template_df.drop_duplicates(subset=merge_cols)[merge_cols]

df_with_agg = pd.merge(AggregatedScores, template_subset, on=['Source', 'Type', 'Model'], how='left')

# Run sensitivity analysis for each hyperparameter
summary_list, detail_list = [], []
for hp in ParamCols:
    summary, detail = calculate_sensitivity(df_with_agg, hp, 'ISCOREgm', n_methods=n_methods, n_metrics=n_metrics)
    if summary is not None:
        summary_list.append(summary)
        detail_list.append(detail)

AggSensitivityDic = pd.concat(summary_list, ignore_index=True) if summary_list else pd.DataFrame()
AggSensitivityDetailDic = pd.concat(detail_list, ignore_index=True) if detail_list else pd.DataFrame()

# Individual method sensitivity analysis
MetricTypesKey = ['fft', 'matching_pursuit', 'welch_evo']
IScoreKey = ['ISCOREam', 'ISCOREgm', 'ISCOREhm']

SensitivityDic, SensitivityDetailDic = {}, {}

for mtype in MetricTypesKey:
    df = AnalAccMItableDic[mtype][AnalAccMItableDic[mtype]['Model'].str.contains('SKZFC', na=False)]

    SensitivityDic[mtype] = {}
    SensitivityDetailDic[mtype] = {}

    for iscore_col in IScoreKey:
        summary_list, detail_list = [], []
        for hp in ParamCols:
            summary, detail = calculate_sensitivity(df, hp, iscore_col)
            if summary is not None:
                summary_list.append(summary)
                detail_list.append(detail)

        SensitivityDic[mtype][iscore_col] = pd.concat(summary_list, ignore_index=True) if summary_list else pd.DataFrame()
        SensitivityDetailDic[mtype][iscore_col] = pd.concat(detail_list, ignore_index=True) if detail_list else pd.DataFrame()

# Combine detailed sensitivity results
SensitivityDetailTabs = pd.DataFrame()
for idx, values in SensitivityDetailDic.items():
    SubTab = pd.DataFrame()
    for sub_idx, sub_values in values.items():
        sub_values['IscoreType'] = sub_idx
        SubTab = pd.concat([SubTab, sub_values])
    SubTab['MetricType'] = idx
    SensitivityDetailTabs = pd.concat([SensitivityDetailTabs, SubTab])
    
# =========================
# Output Structure Documentation
# =========================
# AggSensitivityDic:
#   Summary of hyperparameter sensitivity using the aggregated metric across all methods/scores.
#   - Total_Models: number of unique models accumulated across settings for the hyperparameter.
#   - Total_Effective_DataPoints: Total_Models × n_methods × n_metrics (reflects aggregation basis).
#   Provides a single consolidated view of sensitivity.

# AggSensitivityDetailDic:
#   Detailed per-setting statistics using the aggregated metric.
#   - n: number of unique models per (Type, Setting) for this specific hyperparameter.
#   - n_effective: n × n_methods × n_metrics (effective points behind ISCOREgm).
#   - note: human-readable explanation of aggregation basis.
#   Shows how each setting behaves for the overall aggregated score.

# SensitivityDic[mtype][iscore]:
#   Summary sensitivity per method (mtype) and individual score (iscore).

# SensitivityDetailDic[mtype][iscore]:
#   Per-setting detailed stats per method (mtype) and score (iscore).



### Evaluation Summary Tables

In [19]:
# Generate ablation model names from best models
AblationList = []
for Model in BestModels['Model'].unique():
    splitted = Model.split('_')
    splitted[0] = 'SKZ'
    AblationList.append("_".join(splitted))
    splitted[0] = 'FC'
    AblationList.append("_".join(splitted))

# Extract main models with fixed hyperparameters
OnlyMainModels = AnalAccMItableMerged[(AnalAccMItableMerged['Model'].str.contains('SKZFC', na=False)) 
                                     & (AnalAccMItableMerged['Depth']=='1') 
                                     & (AnalAccMItableMerged['Comp']=='800')
                                     & (AnalAccMItableMerged['LatDim']=='30')]

# Extract ablation models
OnlyAblModels = AnalAccMItableMerged[AnalAccMItableMerged['Model'].isin(AblationList)]
OnlyAblModels = OnlyAblModels[['MetricType','Model', 'Source', 'Type', 'Depth', 'LatDim', 'Comp', 
                               'MS', 'MP', 'AC', 'SS', 'RA', 'ISCOREam',	'ISCOREgm',	'ISCOREhm']].reset_index(drop=True)

# Create benchmark comparison table
BenchCompTabs = pd.concat([OnlyMainModels, BenchAnalAccMItableMerged]).reset_index(drop=True)
BenchCompTabs = BenchCompTabs[['MetricType','Model', 'Source', 'Type', 'Depth', 'LatDim', 'Comp', 
                               'MS', 'MP', 'AC', 'SS', 'RA', 'ISCOREam',	'ISCOREgm',	'ISCOREhm']].reset_index(drop=True)

# Create ablation study table
MainForAbl =  BestModels[['MetricType','Model', 'Source', 'Type', 'Depth', 'LatDim', 'Comp', 
            'MS', 'MP', 'AC', 'SS', 'RA', 'ISCOREam', 'ISCOREgm', 'ISCOREhm']]
AblCompTabs = pd.concat([MainForAbl, OnlyAblModels]).reset_index(drop=True)

In [20]:
AblCompTabs

,MetricType,Model,Source,Type,Depth,LatDim,Comp,MS,MP,AC,SS,RA,ISCOREam,ISCOREgm,ISCOREhm
0,fft,SKZFC_ART_1_30_800_Mimic,Mimic,ART,1,30,800,0.999357,0.690328,0.892231,0.605326,0.978336,0.833116,0.817234,0.800748
1,matching_pursuit,SKZFC_ART_1_30_800_Mimic,Mimic,ART,1,30,800,0.999863,0.599362,0.891265,0.614864,0.978336,0.816738,0.796858,0.776506
2,welch_evo,SKZFC_ART_1_30_800_Mimic,Mimic,ART,1,30,800,0.926841,0.864391,0.905096,0.705723,0.978336,0.876078,0.870776,0.865081
3,fft,SKZFC_ART_1_50_800_VitalDB,VitalDB,ART,1,50,800,0.999897,0.800593,0.929979,0.710048,0.965827,0.881269,0.874190,0.866821
4,matching_pursuit,SKZFC_ART_1_50_800_VitalDB,VitalDB,ART,1,50,800,0.999963,0.756961,0.924429,0.691176,0.965827,0.867671,0.858784,0.849611
5,welch_evo,SKZFC_ART_1_50_800_VitalDB,VitalDB,ART,1,50,800,0.920108,0.907863,0.950037,0.835737,0.965827,0.915915,0.914774,0.913603
6,fft,SKZFC_II_1_50_800_Mimic,Mimic,II,1,50,800,0.998083,0.474773,0.885152,0.379862,0.393246,0.626223,0.574635,0.531157
7,matching_pursuit,SKZFC_II_1_50_800_Mimic,Mimic,II,1,50,800,0.999420,0.387818,0.854176,0.189931,0.393246,0.564918,0.477130,0.398158
8,welch_evo,SKZFC_II_1_50_800_Mimic,Mimic,II,1,50,800,0.953541,0.660575,0.905921,0.441437,0.393246,0.670944,0.629763,0.589993
9,fft,SKZFC_II_1_50_800_VitalDB,VitalDB,II,1,50,800,0.990731,0.571912,0.865239,0.431060,0.810148,0.733818,0.702594,0.669538


### Save Summary Tables

In [8]:
BenchCompTabs.to_csv('EvalResults/SummaryTables/BenchCompTabs.csv', index=False) 
AblCompTabs.to_csv('EvalResults/SummaryTables/AblCompTabs.csv', index=False) 
AggSensitivityDetailDic.to_csv('EvalResults/SummaryTables/AggSensitivityDetail.csv', index=False) 
SensitivityDetailTabs.to_csv('EvalResults/SummaryTables/SensitivityDetailTabs.csv', index=False) 
SenseAccMItable.to_csv('EvalResults/SummaryTables/SenseAccMItable.csv', index=False) 